In [1]:
!pip install psaw

In [2]:
# Imports and options setting
import datetime as dt
import numpy as np
import pandas as pd
from psaw import PushshiftAPI
api = PushshiftAPI()

## Instructions

This will pull all data from the specified start_date to today for a given subreddit. Just enter your start date, the subreddit you want to pull from and the desired output folder.

It's wise to first pull data from two days ago to get a volume baseline estimate. Data volume is highly dependent on the popularity of the subreddit.


In [11]:
start_date = '2021-06-10 START DATE' ## YYYY-MM-DD
subreddit_to_pull = "SEARCH TERMS" #name of subreddit
output_folder = "FILE PATH FOR RESULTS"

In [12]:
def pushshift_to_dataframe(**kwargs):    
    """
    Searches pushshift comments and submissions and returns a pandas dataframe
    
    Example:
    df = pushshift_to_dataframe(subreddit=['metacanada','canadapolitics'], 
                            limit = 10, 
                            filter = ['body','title','selftext','subreddit','author','created_utc','score','controversiality','url','permalink'],
                            min_date = str_date_to_epoch('2019-11-01'), 
                            max_date = str_date_to_epoch('2019-11-02'))
    """
    if 'filter' in kwargs:
        columns_to_get = kwargs['filter']
    else:
        columns_to_get = ['body','title','selftext','subreddit','author','created_utc','score','controversiality','url','permalink','link_id','parent_id','id'],
    # Search for submissions
    # Example URL: https://api.pushshift.io/reddit/submission/search/?after=1h
    submissions = list(api.search_submissions(**kwargs))
    if len(submissions)>0:
        submissions_df = pd.DataFrame([i.d_ for i in submissions])
        submissions_df['post_type']='submission'
        if 'selftext' in columns_to_get:
            submissions_df['text'] = submissions_df['selftext'].fillna('')
            submissions_df.drop(columns=['selftext'],inplace=True)
        if 'id' in columns_to_get:
            submissions_df['id'] = 't3_'+submissions_df['id']
    # Search for comments
    # Example URL: https://api.pushshift.io/reddit/submission/search/?after=1h
    comments = list(api.search_comments(**kwargs))
    if len(comments)>0:
        comments_df = pd.DataFrame([i.d_ for i in comments])
        comments_df['post_type']='comment'
        if 'id' in columns_to_get:
            comments_df['id'] = 't1_'+comments_df['id']
        if 'body' in columns_to_get:
            comments_df['text'] = comments_df['body'].fillna('')
            comments_df.drop(columns=['body'],inplace=True)
    # Merge comments and submissions, if any were found
    if len(submissions)>0:
        if len(comments)>0:
            df = pd.concat([submissions_df, comments_df],ignore_index=True,sort=False)
        else:
            df = submissions_df
    else:
        #If there were no comments or submissions, return empty dataframe
        temp = columns_to_get
        if 'body' in columns_to_get or 'selftext' in columns_to_get:
            temp.append('text')
        if 'body' in columns_to_get:
            temp.remove('body')
        if 'selftext' in columns_to_get:
            temp.remove('selftext')
        return(pd.DataFrame([],columns=temp))
    if 'created_utc' in columns_to_get:
        df['created_utc'] = pd.to_datetime(df['created_utc'],unit='s')
    if 'permalink' in columns_to_get:
        df['permalink'] = 'https://reddit.com'+df['permalink']
    return(df)

def date_to_epoch(date_string):
    return(int(date_string.timestamp()))

def str_date_to_epoch(date_string):
    return(int(dt.datetime.strptime(date_string,'%Y-%m-%d').timestamp()))

In [13]:
df = pushshift_to_dataframe(subreddit=[subreddit_to_pull], 
                            filter = ['body','title','selftext','subreddit','author','created_utc','score','controversiality','url','permalink'],
                            #limit = 100,
                            after = str_date_to_epoch(start_date))

In [14]:
df.to_csv(output_folder+subreddit_to_pull+".csv")